In [18]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from dotenv import load_dotenv
import os

In [19]:
load_dotenv()

client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [13]:
def get_artist_id(artist_name):
    if artist_name.lower() == 'adele':
        return '4dpARuHxo51G3z768sgnrY'
    else:
        results = sp.search(q=f'artist:{artist_name}', type='artist', limit=10)
        items = results['artists']['items']
        
        if len(items) > 0:
            for artist in items:
                if artist['name'].lower() == artist_name.lower():
                    return artist['id']
        return None

In [14]:
def get_albums(artist_id):
    albums = []
    results = sp.artist_albums(artist_id, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    return albums

In [15]:
def get_album_tracks(album_id):
    tracks = []
    results = sp.album_tracks(album_id)
    tracks.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [16]:
def get_track_popularity(track_id):
    track = sp.track(track_id)
    return track['popularity']

def get_audio_features(track_id):
    features = sp.audio_features(track_id)[0]
    return features if features else {}

def get_genres(artist_id):
    artist = sp.artist(artist_id)
    return artist['genres']

def get_adele_songs_data(artist_name):
    artist_id = get_artist_id(artist_name)
    if not artist_id:
        print(f"Artist '{artist_name}' not found.")
        return None
    
    albums = get_albums(artist_id)
    all_tracks = []

    for album in albums:
        album_tracks = get_album_tracks(album['id'])
        for track in album_tracks:
            audio_features = get_audio_features(track['id'])
            track_data = {
                'album_name': album['name'],
                'track_name': track['name'],
                'track_id': track['id'],
                'release_date': album['release_date'],
                'popularity': get_track_popularity(track['id']),
                'duration_ms': audio_features.get('duration_ms'),
                'danceability': audio_features.get('danceability'),
                'energy': audio_features.get('energy'),
                'key': audio_features.get('key'),
                'loudness': audio_features.get('loudness'),
                'mode': audio_features.get('mode'),
                'speechiness': audio_features.get('speechiness'),
                'valence': audio_features.get('valence'),
                'track_url': track['external_urls']['spotify'],
                'album_cover_url': album['images'][0]['url'] if album['images'] else None,
            }
            all_tracks.append(track_data)
    
    if all_tracks:
        return pd.DataFrame(all_tracks)
    else:
        print(f"No tracks found for artist '{artist_name}'.")
        return None

In [17]:
artist_name = "Adele"
adele_songs_data = get_adele_songs_data(artist_name)

if adele_songs_data is not None:
    adele_songs_data.to_csv('adele_songs_data.csv', index=False)
    print(adele_songs_data.head())
else:
    print("No data to save.")

  album_name           track_name                track_id release_date  \
0         30  Strangers By Nature  13CVSGLSFl4UxpDVR6u3dq   2021-11-19   
1         30           Easy On Me  46IZ0fSY2mpAiktS3KOqds   2021-11-19   
2         30       My Little Love  2DuPBbS5mIldXnh7Wum8Cy   2021-11-19   
3         30   Cry Your Heart Out  09u787BYeYIGd2mFIJ505t   2021-11-19   
4         30            Oh My God  3Kkjo3cT83cw09VJyrLNwX   2021-11-19   

   popularity  duration_ms  danceability  energy  key  loudness  mode  \
0          62       182163         0.387   0.253    5    -8.901     1   
1          84       224695         0.604   0.366    5    -7.519     1   
2          64       389107         0.339   0.492    1   -10.661     1   
3          63       255278         0.756   0.713    7    -6.917     1   
4          75       225149         0.534   0.725    1    -3.906     1   

   speechiness  valence                                          track_url  \
0       0.0336    0.134  https://open.